In [ ]:
pip install pyvi

In [ ]:
from pyvi import ViTokenizer
import nltk
import numpy as np

import keras
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import LSTM

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers , activations , models , preprocessing , utils
import tensorflow as tf
from  sklearn.model_selection import train_test_split

from numpy import array
from numpy import argmax
from numpy import array_equal
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input
from keras.layers import TimeDistributed

# Tập data chatbot

In [ ]:
file_tranning = ['bạn bè.txt',
'các câu hỏi phức tạp.txt',
'đất nước.txt',
'địa chỉ.txt',
'du lịch.txt',
'gia đình.txt',
'giải trí.txt',
'học tập.txt',
'nghề nghiệp.txt',
'nghỉ lễ.txt',
'người yêu.txt',
'robot.txt',
'shoping.txt',
'tán gẫu.txt',
'tdtu.txt',
'thông tin cá nhân.txt',
'trò chuyện về đi ăn.txt']

In [ ]:
question_train = []
answer_train = []

for i in range(len(file_tranning)):
    with open(file_tranning[i], encoding='UTF-8') as f:
        train_lines = f.readlines()
        for line in train_lines:
            tmp = line.split("__eou__")
            question_train.append(tmp[0].strip()) # strip(): Loại bỏ whitespace đầu cuối string
            answer_train.append(tmp[1].strip())

In [ ]:
(question_train[:20])
# print((answer_train[0]))

# Preprocessing

In [ ]:
# Xóa question và answer khi answer empty string
list_index_empty_answer = []

In [ ]:
for i in range(len(answer_train)):
    if(answer_train[i] == ""):
        list_index_empty_answer.append(i)

In [ ]:
print(list_index_empty_answer)

In [ ]:
for i in range(len(list_index_empty_answer)):
    del answer_train[list_index_empty_answer[i]-i]
    del question_train[list_index_empty_answer[i]-i]

In [ ]:
import string

In [ ]:
def text_process(mess):
    # chuyển về chữ thường
    mess = mess.lower()
    
    # xóa dấu câu
    mess = [char for char in mess if char not in string.punctuation]
    mess = ''.join(mess)
    
    # replace whitespace
    mess = mess.replace("   ", " ")
    mess = mess.replace("  ", " ")
    
    # Word Segmentation
    mess = ViTokenizer.tokenize(mess)
    
    return mess

In [ ]:
for i in range(len(question_train)):
    question_train[i] = text_process(question_train[i])
    answer_train[i] = text_process(answer_train[i])

In [ ]:
#print((question_train[631]))
#print((answer_train[631]))

In [ ]:
print(question_train[0])
print(answer_train[0])

In [ ]:
data = question_train
data_answer = answer_train

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(lower=True,filters='!"#$%&()*+,-./:;<=>?@[\]^`{|}~ ',split=' ')

tokenizer.fit_on_texts(data)
print(data[0])
print(data[1])
print(data[2])

tokenizer.fit_on_texts(data_answer)
print(data_answer[0])
print(data_answer[1])
print(data_answer[2])

In [ ]:
word2count = {}

for line in data:
    for word in line.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1
for line in data_answer:
    for word in line.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1

In [ ]:
len(word2count)

In [ ]:
thresh = 1

vocab = {}
word_num = 0
for word, count in word2count.items():
    if count >= thresh:
        vocab[word] = word_num
        word_num += 1

In [ ]:
len(vocab)

In [ ]:
for i in range(len(data_answer)):
    data_answer[i] = '<SOS> ' + data_answer[i] + ' <EOS>'


In [ ]:
len(data_answer)

In [ ]:
tokens = ['<PAD>', '<EOS>', '<OUT>', '<SOS>']
x = len(vocab)
for token in tokens:
    vocab[token] = x
    x += 1

In [ ]:
len(vocab)

In [ ]:
vocab['cameron'] = vocab['<PAD>']
vocab['<PAD>'] = 0

In [ ]:
inv_vocab = {w:v for v, w in vocab.items()}
len(inv_vocab)

In [ ]:
encoder_inp = []
for line in data:
    lst = []
    for word in line.split():
        if word not in vocab:
            lst.append(vocab['<OUT>'])
        else:
            lst.append(vocab[word])
        
    encoder_inp.append(lst)

In [ ]:
len(encoder_inp)

In [ ]:
decoder_inp = []
for line in data_answer:
    lst = []
    for word in line.split():
        if word not in vocab:
            lst.append(vocab['<OUT>'])
        else:
            lst.append(vocab[word])        
    decoder_inp.append(lst)

In [ ]:
len(decoder_inp)

In [ ]:
encoder_inp = pad_sequences(encoder_inp, 15, padding='post', truncating='post')
decoder_inp = pad_sequences(decoder_inp, 15, padding='post', truncating='post')


In [ ]:
decoder_final_output = []
for i in decoder_inp:
    decoder_final_output.append(i[1:]) 


In [ ]:
decoder_final_output[:2]

In [ ]:
decoder_final_output = pad_sequences(decoder_final_output, 15, padding='post', truncating='post')
decoder_final_output[:2]

In [ ]:
decoder_final_output = to_categorical(decoder_final_output, len(vocab))

In [ ]:
print(decoder_final_output.shape)

In [ ]:
enc_inp = Input(shape=(15, ))
dec_inp = Input(shape=(15, ))

In [ ]:
VOCAB_SIZE = len(vocab)
embed = Embedding(VOCAB_SIZE+1, output_dim=64, 
                  input_length=15,
                  trainable=True                  
                  )

In [ ]:
enc_embed = embed(enc_inp)
enc_lstm = LSTM(128, return_sequences=True, return_state=True)
enc_op, h, c = enc_lstm(enc_embed)
enc_states = [h, c]

In [ ]:
dec_embed = embed(dec_inp)
dec_lstm = LSTM(128, return_sequences=True, return_state=True)
dec_op, _, _ = dec_lstm(dec_embed, initial_state=enc_states)

dense = Dense(VOCAB_SIZE, activation='softmax')

dense_op = dense(dec_op)

model = Model([enc_inp, dec_inp], dense_op)

model.compile(loss='categorical_crossentropy',metrics=['acc'],optimizer='adam')

model.fit([encoder_inp, decoder_inp],decoder_final_output,epochs=150,batch_size=32)

In [ ]:
enc_model = Model([enc_inp], enc_states)

decoder_state_input_h = Input(shape=(128,))
decoder_state_input_c = Input(shape=(128,))

decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = dec_lstm(dec_embed , 
                                    initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]

dec_model = Model([dec_inp]+ decoder_states_inputs,
                                      [decoder_outputs]+ decoder_states)

In [ ]:
print("##########################################")
print("#       start chatting        #")
print("##########################################")

prepro1 = ""
while prepro1 != 'q':
  prepro1  = input("you : ")
  prepro1 = text_process(prepro1)
  prepro = [prepro1]
  txt = []
  for x in prepro:
    lst = []
    for y in x.split():
      try:
        lst.append(vocab[y])
      except:
        lst.append(vocab['<OUT>'])
    txt.append(lst)
    txt = pad_sequences(txt, 15, padding='post')

    stat = enc_model.predict( txt )
    empty_target_seq = np.zeros( ( 1 , 1) )
    empty_target_seq[0, 0] = vocab['<SOS>']
    stop_condition = False

    decoded_translation = ''

    while not stop_condition :
      dec_outputs , h, c= dec_model.predict([empty_target_seq] + stat )
      decoder_concat_input = dense(dec_outputs)
      sampled_word_index = np.argmax( decoder_concat_input[0, -1, :] )
      sampled_word = inv_vocab[sampled_word_index] + ' '

      if sampled_word != '<EOS> ':
        decoded_translation += sampled_word
      if sampled_word == '<EOS> ' or len(decoded_translation.split()) > 13:
        stop_condition = True

      empty_target_seq = np.zeros( ( 1 , 1 ) )  
      empty_target_seq[ 0 , 0 ] = sampled_word_index

      stat = [h, c]
    print("chatbot attention : ", decoded_translation )
    print("==============================================")  